<a href="https://colab.research.google.com/github/seonghaiiiii/recommendation-systems/blob/main/%EA%B0%9C%EC%9D%B8%EB%B3%84_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
import numpy as np
from surprise import SVD, accuracy
from surprise import Reader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame({
    'user' : ['김땡땡','이땡땡','박땡땡','구땡땡','문땡땡','황땡땡','장땡땡','강땡땡','서땡땡','권땡떙','김땡땡'],
    'item' : ['냉장고','컴퓨터','노트북','떡볶이','필기구','커피','자동차','자동차','돈까스','노트북','아이폰'],
    'score' : [2,3,1,5,1,4,3,2,1,4,3]
})
df

,user,item,score
0,김땡땡,냉장고,2
1,이땡땡,컴퓨터,3
2,박땡땡,노트북,1
3,구땡땡,떡볶이,5
4,문땡땡,필기구,1
5,황땡땡,커피,4
6,장땡땡,자동차,3
7,강땡땡,자동차,2
8,서땡땡,돈까스,1
9,권땡떙,노트북,4


구매한 목록과 평점

In [ ]:
df_cross = pd.crosstab(df['user'],df['item'])
df_cross

item,냉장고,노트북,돈까스,떡볶이,아이폰,자동차,커피,컴퓨터,필기구
user,,,,,,,,,
강땡땡,0,0,0,0,0,1,0,0,0
구땡땡,0,0,0,1,0,0,0,0,0
권땡떙,0,1,0,0,0,0,0,0,0
김땡땡,1,0,0,0,1,0,0,0,0
문땡땡,0,0,0,0,0,0,0,0,1
박땡땡,0,1,0,0,0,0,0,0,0
서땡땡,0,0,1,0,0,0,0,0,0
이땡땡,0,0,0,0,0,0,0,1,0
장땡땡,0,0,0,0,0,1,0,0,0


crosstab - > 행렬로 나타나짐

In [ ]:
df_groupby = df.groupby([df['user'],df['item']])
df_groupby.sum()

score
user item       
강땡땡  자동차       2
구땡땡  떡볶이       5
권땡떙  노트북       4
김땡땡  냉장고       2
     아이폰       3
문땡땡  필기구       1
박땡땡  노트북       1
서땡땡  돈까스       1
이땡땡  컴퓨터       3
장땡땡  자동차       3
황땡땡  커피        4

user 기준 묶여짐

In [ ]:
relation_df = df_groupby.sum().unstack()
relation_df

score                                        
item   냉장고  노트북  돈까스  떡볶이  아이폰  자동차   커피  컴퓨터  필기구
user                                              
강땡땡    NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN  NaN
구땡땡    NaN  NaN  NaN  5.0  NaN  NaN  NaN  NaN  NaN
권땡떙    NaN  4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN
김땡땡    2.0  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN
문땡땡    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
박땡땡    NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN
서땡땡    NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN
이땡땡    NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  NaN
장땡땡    NaN  NaN  NaN  NaN  NaN  3.0  NaN  NaN  NaN
황땡땡    NaN  NaN  NaN  NaN  NaN  NaN  4.0  NaN  NaN

unstack을 통해 0,1이 아닌 score값이 매핑이 됨

In [ ]:
pointer_reader = Reader(rating_scale=(1,5))
load_now_df_data = Dataset.load_from_df(df=df, reader = pointer_reader)

train_set = load_now_df_data.build_full_trainset()
test_set = train_set.build_testset()

model = SVD(n_factors = 100, n_epochs = 30, random_state = 120)
model.fit(train_set)

SVD 행렬 알고리즘을 이용

In [ ]:
pred_test = model.test(test_set)
accuracy.rmse(pred_test)

RMSE: 0.6790


0.6790276663241138

rmse : 큰값들(격차가 심한 데이터)에 민감하게 반응

#user에게 추천해보기

In [ ]:
username = '구땡땡'
items = df['item'].to_numpy()
points = 0

for i in range(len(items)):
  print(model.predict(username,items[i],points))

user: 구땡땡        item: 냉장고        r_ui = 0.00   est = 2.89   {'was_impossible': False}
user: 구땡땡        item: 컴퓨터        r_ui = 0.00   est = 3.01   {'was_impossible': False}
user: 구땡땡        item: 노트북        r_ui = 0.00   est = 2.92   {'was_impossible': False}
user: 구땡땡        item: 떡볶이        r_ui = 0.00   est = 3.77   {'was_impossible': False}
user: 구땡땡        item: 필기구        r_ui = 0.00   est = 2.62   {'was_impossible': False}
user: 구땡땡        item: 커피         r_ui = 0.00   est = 2.72   {'was_impossible': False}
user: 구땡땡        item: 자동차        r_ui = 0.00   est = 2.82   {'was_impossible': False}
user: 구땡땡        item: 자동차        r_ui = 0.00   est = 2.82   {'was_impossible': False}
user: 구땡땡        item: 돈까스        r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 구땡땡        item: 노트북        r_ui = 0.00   est = 2.92   {'was_impossible': False}
user: 구땡땡        item: 아이폰        r_ui = 0.00   est = 2.89   {'was_impossible': False}


떡볶이를 구매한 구땡땡, 무슨 아이템을 추천하는게 좋을까 (떡볶이는 제외)

추천 1 : 자동차

추천 2 : 노트북

추천 3 : 냉장고, 아이폰
.

.

.

In [ ]:
username = '문땡땡'
items = df['item'].to_numpy()
points = 0

for i in range(len(items)):
  print(model.predict(username,items[i],points))

user: 문땡땡        item: 냉장고        r_ui = 0.00   est = 2.24   {'was_impossible': False}
user: 문땡땡        item: 컴퓨터        r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 문땡땡        item: 노트북        r_ui = 0.00   est = 2.46   {'was_impossible': False}
user: 문땡땡        item: 떡볶이        r_ui = 0.00   est = 2.67   {'was_impossible': False}
user: 문땡땡        item: 필기구        r_ui = 0.00   est = 1.66   {'was_impossible': False}
user: 문땡땡        item: 커피         r_ui = 0.00   est = 2.67   {'was_impossible': False}
user: 문땡땡        item: 자동차        r_ui = 0.00   est = 2.32   {'was_impossible': False}
user: 문땡땡        item: 자동차        r_ui = 0.00   est = 2.32   {'was_impossible': False}
user: 문땡땡        item: 돈까스        r_ui = 0.00   est = 2.30   {'was_impossible': False}
user: 문땡땡        item: 노트북        r_ui = 0.00   est = 2.46   {'was_impossible': False}
user: 문땡땡        item: 아이폰        r_ui = 0.00   est = 2.37   {'was_impossible': False}


필기구를 구매한 문땡땡, 무슨 아이템을 추천하는게 좋을까 (필기구는 제외)

추천 1 : 컴퓨터

추천 2 : 떡볶이, 커피

추천 3 : 노트북
.

.

.